In [56]:
from mesa import Model
from mesa import Agent
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random

In [90]:
class AmongUs(Model):
    def __init__(self, map, n_crew, n_impo):
        super().__init__()

        # generate grid
        # self.height = 114
        # self.width = 114
        self.height = 60
        self.width = 60
        self.grid = MultiGrid(self.width, self.height, torus=True)

        # create schedulers for agents
        self.schedule_Crewmate = RandomActivation(self)
        
        # initialize agents
        self.n_crew = n_crew
        self.n_impo = n_impo
        self.spawn_players()

    def spawn_players(self):
        # generate crewmates
        for _ in range(self.n_crew):
            self.new_agent(Crewmate, (5, 5))

        # generate imposters
        for _ in range(self.n_impo):
            self.new_agent(Imposter, (5, 5))

    def new_agent(self, agent_type, pos):
        '''
        Method that creates a new agent, and adds it to the correct scheduler.
        '''
        agent = agent_type(self.next_id(), self, pos)

        self.grid.place_agent(agent, pos)
        getattr(self, f'schedule_{agent_type.__name__}').add(agent)

        
    def step(self):
        '''
        Method that steps every agent. 
        
        Prevents applying step on new agents by creating a local list.
        '''

        self.schedule_Crewmate.step()

        # self.datacollector.collect(self)

    def play_match(self, step_count=50):
        '''
        Method that runs the model for a specific amount of steps.
        '''
        for i in range(step_count):
            self.step()


In [91]:
class Crewmate(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos

    def step(self):
        '''
        This method should get the neighbouring cells (Moore's neighbourhood), select one, and move the agent           to this cell.
        '''
        grid = self.model.grid
        neighborhood = grid.get_neighborhood(self.pos, True)
        grid.move_agent(self, random.choice(neighborhood))

In [92]:
amongUs = AmongUs('map', 3, 0)

amongUs.play_match()